In [1]:
import json
import tkinter as tk
import numpy as np

In [2]:
with open("geoBoundariesCGAZ_ADM0.geojson", "r") as f:
    geo_dict = json.load(f)

features = geo_dict["features"]
countries = []
for feature in features:
    countries.append(feature["properties"]["shapeName"])

In [20]:
root = tk.Tk()
root.title("Country Displayer")
screen_width, screen_height = 700, 700
canvas = tk.Canvas(root, width=screen_width, height=screen_height, bg="black")
canvas.grid(row=0, column=0, columnspan=4)

width_var = tk.IntVar(root, 300)
country_var = tk.StringVar(root, "Afghanistan")

def drawCountry():
    w = width_var.get()
    country = country_var.get()
    idx = countries.index(country)
    geometry = features[idx]["geometry"]

    canvas.delete("all")
    if geometry["type"] == "Polygon":
        points = np.array(geometry["coordinates"][0]).flatten()
        points[1::2] *= -1  # Reversing y points for coordinate change
        init_w = np.max(points[::2]) - np.min(points[::2])
        init_h = np.max(points[1::2]) - np.min(points[1::2])
        scale = w / init_w
        points[::2] = (points[::2] - np.min(points[::2]) - init_w / 2) * scale + screen_width / 2
        points[1::2] = (points[1::2] - np.min(points[1::2]) - init_h / 2) * scale + screen_height / 2
        canvas.create_polygon(*points, fill="white", outline="black")
    elif geometry["type"] == "MultiPolygon":
        max_val = 0
        max_idx = -1
        for idx, poly in enumerate(geometry["coordinates"]):
            points = np.array(poly[0]).flatten()
            if len(points) > max_val:
                max_val = len(points)
                max_idx = idx
        mainland_points = np.array(geometry["coordinates"][max_idx][0]).flatten()
        mainland_points[1::2] *= -1  # Reversing y points for coordinate change
        min_x = np.min(mainland_points[::2])
        min_y = np.min(mainland_points[1::2])
        init_w = np.max(mainland_points[::2]) - min_x
        init_h = np.max(mainland_points[1::2]) - min_y
        scale = w / init_w
        for poly in geometry["coordinates"]:
            points = np.array(poly[0]).flatten()
            points[1::2] *= -1  # Reversing y points for coordinate change
            points[::2] = (points[::2] - min_x - init_w / 2) * scale + screen_width / 2
            points[1::2] = (points[1::2] - min_y - init_h / 2) * scale + screen_height / 2
            canvas.create_polygon(*points, fill="white", outline="black")

def onSlide(val):
    width_var.set(int(val))
    drawCountry()

def onSelect(val):
    country_var.set(val)
    drawCountry()

slider = tk.Scale(root, from_=50, to_=500, length=200, command=onSlide, orient=tk.HORIZONTAL)
slider.set(300)
slider.grid(row=1, column=2, pady=5)

dropdown = tk.OptionMenu(root, country_var, *countries, command=onSelect)
dropdown.grid(row=1, column=1, pady=5)

root.mainloop()